<a href="https://colab.research.google.com/github/ihagoSantos/natural-language-processing/blob/main/data_acquisition_with_data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aquisição de dados com raspagem de dados

In [2]:
# Instalação de pacotes para raspagem e análise de dados
!pip install beautifulsoup4 pandas requests

In [3]:
# Declara funções utilitárias utilizadas no notebook
import datetime
def formata_msg(nivel, msg):
  """
  Formata mensagem de log definindo nível de severidade, timestamp e a mensagem.

  Parâmetros:
  - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING')
  - msg (str): A mensagem de log propriamente dita

  Retorna:
  - str: A mensagem de log formatada
  """
  timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%M')
  return f"[{nivel}] {timestamp} - {msg}"


In [4]:
# Importa os módulos essenciais para o notebook
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

print(formata_msg("INFO", "Bibliotecas importadas com sucesso; ambiente pronto."))

[INFO] 2024-11-22 10:26:26 - Bibliotecas importadas com sucesso; ambiente pronto.


In [5]:
corpora_caminho = './corpora'

if not os.path.exists(corpora_caminho):
  os.mkdir(corpora_caminho)
  print(formata_msg('INFO', f'Diretório {corpora_caminho} criado com sucesso!'))
else:
  print(formata_msg('INFO', f'Diretório {corpora_caminho} já existe no sistema!'))

[INFO] 2024-11-22 10:31:31 - Diretório ./corpora criado com sucesso!


# 1. G1

## Raspagem de dados de notícias do G1

Os elementos estruturais da página principal, essenciais para a raspagem de dados incluem as seguintes classes CSS:
- feed-post-body: corpo da notícia
- feed-post-link: título da notícia
- feed-post-body-resumo: subtítulo da notícia (opcional)

In [7]:
url = 'https://g1.globo.com'

In [29]:
noticias_arquivo = 'g1_noticias.csv'

noticias_caminho = os.path.join(corpora_caminho, noticias_arquivo)

noticias_g1 = []

resposta = requests.get(url)

if resposta.status_code == 200:
  pagina = resposta.content
  soup = BeautifulSoup(pagina, 'html.parser')

  # encontra todos os elementos que correspondem ao corpo da notícia
  noticias = soup.find_all('div', attrs={'class': 'feed-post-body'})

  for noticia in noticias:
    print(noticia)
    # Acha o título da notícia
    titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

    # Tenta achar o subtítulo da notícia
    subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

    noticias_g1.append([
        titulo.text if titulo and titulo.text else '',
        subtitulo.text if subtitulo and subtitulo.text else '',
        titulo.attrs['href'] if titulo.has_attr('href') else ''
    ])


  # Converte a lista de notícias em um dataframe para facilitar a manipulação
  noticias_g1_df = pd.DataFrame(noticias_g1, columns=['titulo', 'subtitulo', 'link'])

  # Salva os dados coletados em um arquivo csv
  noticias_g1_df.to_csv(noticias_caminho, index=False)

  print(formata_msg('INFO', f'Dados coletados com sucesso na URL: {url};'))
  print(formata_msg('INFO', f'Os resultados foram salvos no arquivo: {noticias_caminho}.'))
else:
  print(formata_msg('ERROR', f'Falha na solicitação, código de status: {resposta.status_code}.'))

<div class="feed-post-body"><div class="feed-post-header"></div><div class="feed-post-body-title gui-color-primary gui-color-hover"><div class="_evt"><h2><a class="feed-post-link gui-color-primary gui-color-hover" href="https://g1.globo.com/politica/blog/natuza-nery/post/2024/11/22/indiciamento-na-tentativa-de-golpe-e-o-mais-problematico-para-bolsonaro-e-situacao-do-ex-presidente-piora-ainda-mais.ghtml"><p elementtiming="text-ssr">NATUZA: PF coloca Bolsonaro como articulador do golpe</p></a></h2></div></div><div class="feed-media-wrapper"><a class="feed-post-figure-link gui-image-hover" href="https://g1.globo.com/politica/blog/natuza-nery/post/2024/11/22/indiciamento-na-tentativa-de-golpe-e-o-mais-problematico-para-bolsonaro-e-situacao-do-ex-presidente-piora-ainda-mais.ghtml"><div class="bstn-fd-item-cover"><picture class="bstn-fd-cover-picture"><img alt="NATUZA: PF coloca Bolsonaro como articulador do golpe - Foto: (REUTERS/Adriano Machado)" class="bstn-fd-picture-image" elementtiming

In [30]:
# Carrega os dados do arquivo csv especificados no DataFrame 'noticias_g1_df'
noticias_g1_df = pd.read_csv(noticias_caminho)
noticias_g1_df.head(10)

,titulo,subtitulo,link
0,NATUZA: PF coloca Bolsonaro como articulador d...,NaN,https://g1.globo.com/politica/blog/natuza-nery...
1,"Heleno, Braga Netto: o que pode acontecer com ...",NaN,https://g1.globo.com/politica/noticia/2024/11/...
2,Inquérito do golpe,NaN,NaN
3,Governo deve anunciar hoje novo bloqueio no Or...,NaN,https://g1.globo.com/economia/noticia/2024/11/...
4,Não é um bônus! Relembre como surgiu o 13° sal...,NaN,https://g1.globo.com/economia/educacao-finance...
5,Qual a chance de Netanyahu ser preso (e como T...,NaN,https://g1.globo.com/mundo/blog/sandra-cohen/p...
6,"Cobrado por pais, Tarcísio lamenta morte de es...",NaN,https://g1.globo.com/sp/sao-paulo/noticia/2024...
7,Mulher é baleada dentro de carro no Rio; famíl...,NaN,https://g1.globo.com/rj/rio-de-janeiro/noticia...
8,Vídeos curtos do g1,NaN,NaN
